In [1]:
DATA_NAME = 'segmentAnything-full' 
TRANSFORM = 'wavelet-diagonal'
CHANNEL = 'green'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_green_diagonal_wavelet_full_segmentAnything.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10
2,29108.11,-0.17,-0.02,0.10,0.00,-0.01,0.0,-0.00,-0.00
3,-0.17,6762.88,-0.04,0.02,-0.02,0.00,0.0,0.00,0.00
4,-0.02,-0.04,1463.98,0.02,-0.00,-0.00,-0.0,-0.00,0.00
5,0.10,0.02,0.02,296.07,0.00,-0.00,0.0,0.00,0.00
6,0.00,-0.02,-0.00,0.00,56.90,-0.00,-0.0,0.00,0.00
7,-0.01,0.00,-0.00,-0.00,-0.00,10.57,0.0,0.00,0.00
8,0.00,0.00,-0.00,0.00,-0.00,0.00,1.9,0.00,0.00
9,-0.00,0.00,-0.00,0.00,0.00,0.00,0.0,0.31,-0.00
10,-0.00,0.00,0.00,0.00,0.00,0.00,0.0,-0.00,0.04


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10
2,1.00000,-0.00001,-0.00000,0.00003,0.00000,-0.00002,0.00001,-0.00001,-0.00000
3,-0.00001,1.00000,-0.00001,0.00001,-0.00003,0.00001,0.00004,0.00000,0.00000
4,-0.00000,-0.00001,1.00000,0.00003,-0.00000,-0.00000,-0.00003,-0.00001,0.00001
5,0.00003,0.00001,0.00003,1.00000,0.00001,-0.00002,0.00005,0.00000,0.00000
6,0.00000,-0.00003,-0.00000,0.00001,1.00000,-0.00001,-0.00001,0.00002,0.00001
7,-0.00002,0.00001,-0.00000,-0.00002,-0.00001,1.00000,0.00002,0.00000,0.00000
8,0.00001,0.00004,-0.00003,0.00005,-0.00001,0.00002,1.00000,0.00003,0.00000
9,-0.00001,0.00000,-0.00001,0.00000,0.00002,0.00000,0.00003,1.00000,-0.00001
10,-0.00000,0.00000,0.00001,0.00000,0.00001,0.00000,0.00000,-0.00001,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.2956234872235596

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[9.42302503e+07 5.00326780e+06 2.29672489e+05 9.13461052e+03
 3.23985752e+02 1.04987478e+01 3.01311250e-01 6.24191248e-03
 0.00000000e+00]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8
0,0.999510,-0.030652,-0.006249,-0.001257,-0.000242,-0.000045,-0.000008,-0.000001,-1.764973e-07
1,0.030424,0.998985,-0.032606,-0.006251,-0.001202,-0.000222,-0.000039,-0.000007,-8.492661e-07
2,0.007185,0.032153,0.998812,-0.035288,-0.006473,-0.001201,-0.000217,-0.000036,-4.578522e-06
3,0.001686,0.007282,0.034763,0.998538,-0.040067,-0.007132,-0.001274,-0.000212,-2.730591e-05
4,0.000389,0.001677,0.007744,0.039407,0.997989,-0.048284,-0.008306,-0.001373,-1.774016e-04
5,0.000091,0.000388,0.001792,0.008873,0.047335,0.996866,-0.061937,-0.009834,-1.269018e-03
6,0.000021,0.000092,0.000428,0.002107,0.010993,0.060393,0.994429,-0.085007,-1.054173e-02
7,0.000005,0.000023,0.000105,0.000519,0.002703,0.014555,0.082175,0.988059,-1.294881e-01
8,0.000001,0.000006,0.000028,0.000136,0.000708,0.003810,0.021224,0.128119,9.915241e-01


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0004902298715965259,
 0.0010151724144202845,
 0.001187739287476841,
 0.0014621859030768958,
 0.002011046371359071,
 0.003133558745890719,
 0.005570578826469164,
 0.011940974209363242,
 0.008475884307429071]